In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import metrics

# Define the paths to your stone and normal image directories
stone_dir = "/content/drive/MyDrive/Kidney_stone_detection-main/Kidney_stone_detection-main/Dataset/Train/Kidney_stone"
normal_dir = "/content/drive/MyDrive/Kidney_stone_detection-main/Kidney_stone_detection-main/Dataset/Train/Normal"

# Function to load and preprocess images
def load_and_preprocess_images(image_dir, label):
    images = []
    labels = []
    for filename in os.listdir(image_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read the image
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Assuming images are grayscale

            # Normalize pixel values to be between 0 and 1
            image = image / 255.0

            # Resize the images to a standard size (e.g., 128x128)
            image = cv2.resize(image, (128, 128))

            images.append(image.flatten())  # Flatten the image to a 1D array
            labels.append(label)

    return images, labels

# Load and preprocess stone images
stone_images, stone_labels = load_and_preprocess_images(stone_dir, label=1)

# Load and preprocess normal images
normal_images, normal_labels = load_and_preprocess_images(normal_dir, label=0)

# Combine stone and normal data
X = stone_images + normal_images
y = stone_labels + normal_labels

# Convert the lists to NumPy arrays for compatibility with scikit-learn
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()

# Create an SVM classifier
svm_classifier = make_pipeline(scaler, SVC(C=1.0, kernel='rbf', gamma='auto'))

# Train the SVM classifier
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Additional metrics
print("Classification Report:\n", metrics.classification_report(y_test, y_pred))
print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))


Accuracy: 0.9140893470790378
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.99      0.93       166
           1       0.99      0.81      0.89       125

    accuracy                           0.91       291
   macro avg       0.93      0.90      0.91       291
weighted avg       0.92      0.91      0.91       291

Confusion Matrix:
 [[165   1]
 [ 24 101]]
